In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class Data():

  # n = size of data
  # m = number of dimension
  # sigma = The spread of noise in the output variable
  # theta =  The probability of flipping the label, Y

  def __init__(self,n,m):
    
    self.n = n 
    self.m = m
  
  def linear_data(self,sigma):
    '''generating (m +1) data for independent variable(X)'''

    self.sigma = sigma
    #np.random.seed(101)
    X = np.random.normal(size =(self.n,self.m))
    ones = np.ones(X.shape[0],dtype= int)
    X = np.c_[ones,X]

    beta = np.random.normal(size =(self.m+1,1))   # generating random beta values

    mu = 0
    Y = np.dot(X,beta) + np.random.normal(mu,sigma )  #generating dependent variable(Y)

    return X,Y,beta

  def logistic_data(self,theta):
    '''generating (m +1) data for independent variable(X)'''

    self.theta = theta
    #np.random.seed(101)
    X = np.random.normal(size =(self.n,self.m))
    ones = np.ones(X.shape[0],dtype= int)
    X = np.c_[ones,X]
    
    beta = np.random.normal(size =(self.m+1,1))   # generating random beta values

    Z = np.dot(X,beta)    #generating dependent variable(Y)

    Y_ = 1/(1 + np.exp(-Z)) # sigmoid function

    # replacing Y values with 0 and 1
    Y_[Y_ > 0.5] = 1
    Y_[Y_ <= 0.5] = 0

    # adding noise to the data
    Y = np.random.binomial(1,self.theta,Y_.shape[0])
    Y = Y.reshape(Y_.shape[0],Y_.shape[1])

    return X,Y,beta

  def split_data (self,ratio):
    '''ratio  = insert the ratio of the training data you want such as 0.8 or 0.6'''
    self.ratio = ratio
    X_train,X_test = X[:int(self.ratio*X.shape[0]),:], X[int(self.ratio*X.shape[0]):,:]
    Y_train,Y_test = Y[:int(self.ratio*X.shape[0]),:], Y[int(self.ratio*X.shape[0]):,:]

    return X_train,X_test,Y_train,Y_test


In [ ]:
p = Data(10000,2)
X,Y = p.logistic_data(0.9)[0],p.logistic_data(0.9)[1]

In [ ]:
X_train = Data(10000,2).split_data(0.6)[0]
Y_train = Data(10000,2).split_data(0.6)[2]

In [ ]:
class Regression():

  def __init__(self,X_train,Y_train,epoch,threshold,alpha):

    self.X_train = X_train
    self.Y_train = Y_train
    self.epoch = epoch
    self.threshold = threshold
    self.alpha = alpha
  
  def linear_regression(self):
    
    beta = np.ones((self.X_train.shape[1],1))
    Y_pred = np.dot(self.X_train,beta)

    initial_cost = np.mean((self.Y_train - Y_pred)**2)  # cost function

    for i in range(self.epoch):
      derivative_wrt_beta = (1/self.X_train.shape[0])* sum((Y_pred - self.Y_train)* self.X_train)
      beta= beta.T - self.alpha*derivative_wrt_beta

      new_cost = np.mean(self.Y_train - Y_pred)**2

      if initial_cost-new_cost > self.threshold:
        initial_cost = new_cost
      else:
        break  
  
    return beta,new_cost

  def logistic_regression(self):
    
    beta = np.ones((self.X_train.shape[1],1))  #initilizing beta with ones 

    for i in range(self.epoch):
      Z = np.dot(self.X_train,beta)
      Y_pred = 1/(1 + np.exp(-Z))  #calling sigmoid function

      # cost function
      initial_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred))

      #gradient descent
      derivative_wrt_beta = (1/self.X_train.shape[0]) * np.dot((Y_pred-self.Y_train).T,self.X_train) #dervatives w.r.t beta

      # cost function
      new_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred)) 

      beta = beta.T - self.alpha * derivative_wrt_beta

      if (initial_cost - new_cost) > self.threshold:
        initial_cost = new_cost
      else:
        break
   
    return new_cost,beta

  def logistic_regression_l1_regularizaion(self):
    
    beta = np.ones((self.X_train.shape[1],1))  #initilizing beta with ones 

    for i in range(self.epoch):
      Z = np.dot(self.X_train,beta)
      Y_pred = 1/(1 + np.exp(-Z))  #calling sigmoid function

      # cost function
      initial_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred))

      #gradient descent
      derivative_wrt_beta = (1/self.X_train.shape[0]) * np.dot((Y_pred-self.Y_train).T,self.X_train) #dervatives w.r.t beta

      # cost function
      new_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred)) 

      beta = beta.T - self.alpha * derivative_wrt_beta

      if (initial_cost - new_cost) > self.threshold:
        initial_cost = new_cost
      else:
        break
   
    return new_cost,beta

  def logistic_regression_l2_regularizaion(self,lambda_value):
    self.lambda_value = lambda_value
    
    beta = np.ones((self.X_train.shape[1],1))  #initilizing beta with ones 

    for i in range(self.epoch):
      Z = np.dot(self.X_train,beta)
      Y_pred = 1/(1 + np.exp(-Z))  #calling sigmoid function

      l2 = (self.lambda_value / 2 * self.X_train.shape[0]) * np.sum(beta**2)  #l2 regularization 

      # cost function
      initial_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred))

      #gradient descent
      derivative_wrt_beta = (1/self.X_train.shape[0]) * np.dot((Y_pred-self.Y_train).T,self.X_train) #dervatives w.r.t beta

      # cost function
      new_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred)) 

      beta = beta.T - self.alpha * derivative_wrt_beta

      if (initial_cost - new_cost) > self.threshold:
        initial_cost = new_cost
      else:
        break
   
    return new_cost,beta

  
  def logistic_regression_l1_regularizaion(self,lambda_value):
    self.lambda_value = lambda_value
    
    beta = np.ones((self.X_train.shape[1],1))  #initilizing beta with ones 

    for i in range(self.epoch):
      Z = np.dot(self.X_train,beta)
      Y_pred = 1/(1 + np.exp(-Z))  #calling sigmoid function

      l1 = (self.lambda_value / 2 * self.X_train.shape[0]) * np.sum(np.absolute(beta))  #l1 regularization 

      # cost function
      initial_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred)) + l1

      #gradient descent
      derivative_wrt_beta = (1/self.X_train.shape[0]) * np.dot((Y_pred-self.Y_train).T,self.X_train) #dervatives w.r.t beta

      # cost function
      new_cost = -(1/self.X_train.shape[0])*np.sum(self.Y_train*np.log(Y_pred) + (1-self.Y_train)*np.log(1-Y_pred)) 

      beta = beta.T - self.alpha * derivative_wrt_beta

      if (initial_cost - new_cost) > self.threshold:
        initial_cost = new_cost
      else:
        break
   
    return new_cost,beta



In [ ]:
a = Regression(X_train,Y_train,100,0.00001,0.01)
derivative_wrt_beta = a.logistic_regression()
print(derivative_wrt_beta)

(0.5136318523297317, array([[1.00208946, 0.99810413]]))


In [ ]:
data = [{'n':100,'theta':0.9,'beta0':1.0020097,'beta1':0.99707731},
        {'n':100,'theta':0.6,'beta0':0.99857502,'beta1':0.99797398},
        {'n':100,'theta':0.3,'beta0':0.9964897,'beta1':0.99700508},
        {'n':1000,'theta':0.9,'beta0':1.00201624,'beta1':0.99811762},
        {'n':1000,'theta':0.6,'beta0':0.99872765,'beta1':0.99872765},
        {'n':1000,'theta':0.3,'beta0':0.99579561,'beta1':0.99819137},
        {'n':10000,'theta':0.9,'beta0':1.00208946,'beta1':0.99810413},
        {'n':10000,'theta':0.6,'beta0':0.99910981,'beta1':0.99814201},
        {'n':10000,'theta':0.3,'beta0':0.99610203,'beta1':0.99828696}
        ]
df = pd.DataFrame(data)
df

,n,theta,beta0,beta1
0,100,0.9,1.002010,0.997077
1,100,0.6,0.998575,0.997974
2,100,0.3,0.996490,0.997005
3,1000,0.9,1.002016,0.998118
4,1000,0.6,0.998728,0.998728
5,1000,0.3,0.995796,0.998191
6,10000,0.9,1.002089,0.998104
7,10000,0.6,0.999110,0.998142
8,10000,0.3,0.996102,0.998287
